In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd

BASE_URL = "https://himalayas.app"  # Replace with actual base URL
CATEGORY_URL = f"{BASE_URL}/resumes"  # Replace with actual category page URL

response = requests.get(CATEGORY_URL)
print(response.text)  # Check if the HTML contains expected elements

# def get_job_categories():
#     response = requests.get(CATEGORY_URL)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     job_categories = []
#     print(soup.select("a.text-primary-700"))
#     for a_tag in soup.select("a.text-primary-700"):  # More general selector
#         print(a_tag)
#         if 'href' in a_tag.attrs:
#             job_categories.append((a_tag.text.strip(), BASE_URL + a_tag['href']))

#     return job_categories

# def get_cv_data(category_name):
#     """Fetch all text from CVs on a job category page."""
#     response = requests.get(f"https://himalayas.app/resumes/{category_name}")
#     soup = BeautifulSoup(response.text, 'html.parser')
#     print(soup.prettify())
#     cvs = []
#     for cv_div in soup.select("div.flex.h-full.w-full.flex-row.rounded-lg.bg-white"):
#         print(cv_div)
#         text_content = " ".join(cv_div.stripped_strings)  # Extract all text and clean up spaces
#         cvs.append({
#             "Category": category_name,
#             "Text": text_content
#         })
    
#     print(cvs)

#     return cvs

import cloudscraper
scraper = cloudscraper.create_scraper()  # Mimics a real browser

def get_cv_data(category_name):
    """Fetch all text from CVs using Cloudscraper."""
    url = f"https://himalayas.app/resumes/{category_name}"
    response = scraper.get(url)
    
    if response.status_code != 200:
        print("Failed to access:", url)
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup.prettify())
    
    cvs = []
    
    for cv_div in soup.select("div.flex.h-full.w-full.flex-row.rounded-lg.bg-white"):
        text_content = " ".join(cv_div.stripped_strings)
        cvs.append({"Category": category_name, "Text": text_content})

    # print(cvs)
    return cvs


file_path = "job_category_clean.xlsx"  
def main():
    job_categories = pd.read_excel(file_path, header=None, dtype=str).values[5500:].ravel()
    print(job_categories)
    
    with open("cv_text_data10.csv", "w", newline="", encoding="utf-8") as file:
        fieldnames = ["Category", "Text"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for category_name in job_categories:
            print(f"Scraping: {category_name}")
            cvs = get_cv_data(category_name)
            for cv in cvs:
                writer.writerow(cv)
            time.sleep(2)  # Respectful delay to avoid being blocked

if __name__ == "__main__":
    main()


<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background-s

ChunkedEncodingError: Response ended prematurely

In [16]:
import pandas as pd
import glob

# Create a pattern to match all CSV files
file_pattern = "cv_text_data*.csv"

# Get all files matching the pattern
files = glob.glob(file_pattern)

# Initialize an empty list to store dataframes
dfs = []

# Loop through all files and read them into dataframes
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all dataframes into one
concatenated_df = pd.concat(dfs, ignore_index=True)

# Optionally, save the concatenated dataframe to a new CSV
concatenated_df.to_csv("concatenated_cv_text_data.csv", index=False)

print("Files have been concatenated successfully!")


Files have been concatenated successfully!


In [18]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the CSV
df = pd.read_csv("concatenated_cv_text_data.csv")

# Initialize necessary tools
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Preprocessing function
def preprocess_text(text):
    # Remove the literal phrase "Contact [email protected]"
    text = re.sub(r'Contact \[email protected\]', '', text)
    
    # Remove the bullet character '•'
    text = text.replace('•', '')
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization (split into words)
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    # Join the words back into a single string
    return " ".join(words)

# Apply preprocessing to the 'Text' column
df['Processed_Text'] = df['Text'].apply(preprocess_text)

# Optionally, remove rows with missing text
df = df.dropna(subset=['Processed_Text'])

# Save the preprocessed data to a new CSV
df.to_csv("preprocessed_cv_text_data.csv", index=False)

print("Preprocessing complete!")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\84905\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\84905\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\84905\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Preprocessing complete!
